In [1]:
import nltk
import random
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split

In [2]:
with open('BIG_BOT_CONFIG.json','r') as f:
    BOT_CONFIG=json.load(f)

In [3]:
def cleaner(text): 
    cleaned_text = ''
    for ch in text.lower():
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz ':
            cleaned_text = cleaned_text + ch
    return cleaned_text

def match(text, example): 
    return nltk.edit_distance(text, example) / len(example) < 0.4 if len(example) > 0 else False

def get_intent(text): 
    for intent in BOT_CONFIG['intents']:
        if 'examples' in BOT_CONFIG['intents'][intent]:
             for example in BOT_CONFIG['intents'][intent]['examples']:
                if match(cleaner(text), cleaner(example)):
                       return intent

In [4]:
X = []
y = []

for intent in BOT_CONFIG['intents']:
     if 'examples' in BOT_CONFIG['intents'][intent]:
        X += BOT_CONFIG['intents'][intent]['examples']
        y += [intent for i in range(len(BOT_CONFIG['intents'][intent]['examples']))]

In [5]:
vectorizer = CountVectorizer(preprocessor=cleaner, ngram_range=(1,3), stop_words=['а', 'и'])

In [6]:
vectorizer.fit(X)
X_vect = vectorizer.transform(X)

In [7]:
X_train_vect, X_test_vect, y_train, y_test = train_test_split(X_vect, y, test_size=0.3)

In [8]:
sgd = SGDClassifier() 
sgd.fit(X_vect, y)

SGDClassifier()

In [9]:
sgd.score(X_vect, y)

0.7829935622317596

In [10]:
def get_intent_by_model(text): # Функция определяющая интент текста с помощью ML-модели
     return sgd.predict(vectorizer.transform([text]))[0]

In [11]:
def bot(text): # функция бота
    intent = get_intent(text)  # 1. попытаться понять намерение сравнением по Левинштейну
    
    if intent is None:
        intent = get_intent_by_model(text) # 2. попытаться понять намерение с помощью ML-модели

    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
question = ''
while question not in ['выход', 'выключайся']:
    question = input()
    answer = bot(question)
    print(answer)

Привет
И вам хорошего дня
Как дела?
Шалом, юзер
Что делаешь?
До новых встреч!
Что?
Привет, всё хорошо?
Погоди, о чем ты?
Благодарю. Я обязательно передам это своим разработчикам!
Ок
и..? =)
Как жизнь?
I’m OK.
